# Weekly Load Automation
- This file will be the main interface to run python code which will operate on the respective folder used in weekly load. 
- The main function code sits in lib folder which is a python package. The package is structured in a way that each module within works on specific working folder for weekly load.
- The config module in lib package has a variable "IS_DEV" which is used as a flag to determine if the code is running in dev environment or not and gets fed into all modules. please make sure it's turned to "False" before running in production.

In [1]:
import lib
from datetime import date, timedelta
import warnings
warnings.filterwarnings("ignore")
import pandas as pd

pd.options.display.float_format = '{:.2f}'.format

# last week's validation date 
from_date = date(2024, 3, 8)
# new week's date to work on
new_date = from_date + timedelta(days=7)
# check environment, if production it should return share drive path
lib.liquid.BASE_PATH

WindowsPath('S:/IT IRSR Shared/RedSwan/RedSwan/Master_bcIMC/LIQUID/Liquid')

# Liquid
- all code related to operation in liquid folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid 

## create template folder
- the code will copy folder according to `from_date` and paste as `to_date`
- delete unnecessary files from last week
- update enviroment file with new date

In [3]:
lib.liquid.create_template_folder(from_date,new_date)

Copied folder from c:\Users\CXimen\OneDrive - BCI\Documents\Liquid_Data_Production_Dev\2024\02\02_02 to c:\Users\CXimen\OneDrive - BCI\Documents\Liquid_Data_Production_Dev\2024\02\02_09
Copied to folder at c:\Users\CXimen\OneDrive - BCI\Documents\Liquid_Data_Production_Dev\2024\02\02_09
Deleted file: Basket_Hedge_20240202.docx
Deleted file: Basket_Hedge_20240202.xml
Deleted file: FixThree_20240202.docx
Deleted file: FixThree_20240202.xml
Deleted file: FixTwo_20240202.docx
Deleted file: FixTwo_20240202.xml
Deleted file: Fix_20240202.docx
Deleted file: Fix_20240202.xml
Deleted file: IFT_20240202.docx
Deleted file: IFT_20240202.xml
Deleted file: Illiquids_20240202.docx
Deleted file: Illiquids_20240202.xml
Deleted file: Main_20240202.docx
Deleted file: Main_20240202.xml
Deleted file: Basket_Hedge_20240202.rml
Deleted file: Basket_Hedge_20240202_Log.csv
Deleted file: Basket_Hedge_20240202_RawLog.csv
Deleted file: Basket_Hedge_overlapped_20240202_holdings.csv
Deleted file: Basket_Hedge_overl

## create Portfolio Filter Group File
- the code will look for `Portfolio Filter Group` file with the date of `from_date` in the `to_date` working folder. update `to_date` portfolios in the file.
- `to_date` portfolio is obtained from sql file `sql\liquid\portfolio_filter_group.sql`
- save and delete last week's file
- make sure `Portfolio Filter Group` file with the date of `from_date` exist in the folder.

In [13]:
lib.liquid.create_portfolio_filter_group(from_date,new_date)

Created Portfolio Filter Group file at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Portfolio Filter Group 20240229.xlsx
Deleted file: Portfolio Filter Group 20240308.xlsx


## Check Cash Counterparty
- the code will check `CounterParty` against `ENRICHMENT_viwCounterpartyMap` table and return anyone that does not exist in the mds table

In [3]:
lib.liquid.counter_party_check()

All counter party are mapped


## Check SuperD Basket Swap File
- the code will look for file with name `BTRSEQ` in the `to_date` working folder in share drive `T:\EDM\Sample Data\daily_dumps\SuperD`

In [4]:
lib.liquid.get_superD_BTRSEQ(new_date)

,External Trade ID,Volume
External ID,,
SW24043145,329.00,67271972.88
SW24043146,986.00,60681024.11
SW63604303,3.00,2275000.00
SW65965146,5.00,3800000.00
SW66867128,986.00,25658338.72
SW66867155,1167.00,30903371.59
SW67357389,2.00,1000000.00
SW67599405,2.00,2930000.00
Total,3480.00,194519707.31


## download weekly liquid data
- the code will run sql query to get basket_hedge, IFT, illiquids, main from the database and save it in the `file` folder

In [5]:
lib.liquid.save_weekly_liquid_data(new_date)

Saved IFT at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\IFT
S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Positions_20240229_IFT.xlsx is updated and saved to None
Renamed Positions_20240229_IFT.xlsx to Positions_20240229_IFT.xlsx
Saved Main at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Main
S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Positions_20240229_Main.xlsx is updated and saved to None
Renamed Positions_20240229_Main.xlsx to Positions_20240229_Main.xlsx
Saved Illiquids at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Illiquids
S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Positions_20240229_Illiquids.xlsx is updated and saved to None
Renamed Positions_20240229_Illiquids.xlsx to Positions_20240229_Illiquids.xlsx
Saved Basket_Hedge at S:\IT IRSR Shared\RedSwan\RedSwan\

## update position in env file
- the code will update the position in env file `NewArch_LiquidsDerivatives V1 CSV.environment`
- input params are the date of the working folder and position we want to change it to.
- make sure the position passed in the function is in the list below
- [`Basket_Hedge`, `Fix`, `IFT`, `Illiquids`, `Main`, `FixTwo`]

In [10]:
lib.liquid.update_env_file_position(new_date, "FixTwo")

replaced Basket_Hedge|Fix|IFT|Illiquids|Main|FixTwo with FixTwo in S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\NewArch_LiquidsDerivatives V1 CSV.environment


## create fix file
- the code will read main data redswan log file from `result` folder and filter out bond records that is modeled to proxy.
- update the model rule effective column
- and save to `file` folder

In [7]:
lib.liquid.create_fix_file(new_date)

S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files\Positions_20240229_Fix.xlsx is updated and saved to None
Renamed Positions_20240229_Fix.xlsx to Positions_20240229_Fix.xlsx
Saved Positions_20240229_Fix.csv at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\Files


## Compile log file
- the code will read all files containing `_Log.csv` in the `Results` folder, and compile them into `Results/log.csv`
- update the compiled log data into `Log_Check.xlsx`

In [7]:
lib.liquid.compile_log(new_date)

## Update PV Validation File
- the code will read look for `PV Report Liquids.xlsx` in the folder, change the name and read the data in the file.
- look for file name contains `LiquidsDerivatives PV Validation` in the folder and delete old weeks data and update with new data.

In [4]:
lib.liquid.create_pv_validation(new_date)

Renamed PV Report Liquids.xlsx to PV Report Liquids 20240229.xlsx
S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Liquid\2024\02\02_29\LiquidsDerivatives PV Validation 20240223.xlsx is updated and saved to None
Renamed LiquidsDerivatives PV Validation 20240223.xlsx to LiquidsDerivatives PV Validation 20240229.xlsx


# Collteral
- all code related to operation in collateral folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral

## create template folder

In [9]:
lib .collateral.create_template_folder(from_date,new_date)

Copied folder from S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_05 to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_12
Copied to folder at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_12
Deleted file: Collateral.docx
Deleted file: Collateral.xml
Deleted file: Collateral_20240105.rml
Deleted file: Collateral_20240105_Enriched.xml
Deleted file: Collateral_20240105_Log.csv
Deleted file: Collateral_20240105_RawLog.csv
Deleted file: Collateral_unprocessed_holdings.csv
Deleted file: Collateral_unprocessed_securities.csv
Deleted file: report.txt
Deleted file: OTC_Collateral_20240105.csv
replaced S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_05 with S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_12 in S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2024\01\01_12\Loading\OTC_Collateral.environment
replaced 20240105 with 20240112 

## Convert to CSV
- the code will look for `OTC_Collateral_date.xlsx` in the folder and convert it to csv file and save it in the same folder

In [23]:
lib.collateral.convert_to_csv(new_date)

Converted S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Collateral\2023\07\07_07\OTC_Collateral_20230707.xlsx to csv


# Repo
- all code related to operation in repo folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo

## create template folder

In [10]:
lib.repo.create_template_folder(from_date,new_date)

Copied folder from S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_05 to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_12
Copied to folder at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_12
Deleted file: Repo.docx
Deleted file: Repo.rml
Deleted file: Repo.xml
Deleted file: Repo_Enriched.xml
Deleted file: Repo_Log.csv
Deleted file: Repo_RawLog.csv
Deleted file: unprocessed_holdings.csv
Deleted file: unprocessed_securities.csv
Deleted file: REPO FINAL.csv
replaced S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_05 with S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_12 in S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\LIQUID\Repo\2024\01\01_12\Loading\Repo_Col_V2.environment


## Update Excel File
- the code will read look for `REPO FINAL.xlsx` in the folder, delete old weeks data and update sql data in the file.
- the second function will convert the excel to csv.

In [4]:
lib.repo.update_excel_file(new_date)
lib.repo.convert_to_csv(new_date)

ValueError: No data found in database

# Lookthrough for Cube
- all code related to operation in Lookthrough for Cube folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube

## create template folder

In [12]:
lib.cube_lookthru.create_template_folder(from_date,new_date)

Copied folder from S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240105 to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112
Copied to folder at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112
Deleted file: Lookthrough_Cube.docx
Deleted file: Lookthrough_Cube.rml
Deleted file: Lookthrough_Cube.xml
Deleted file: Lookthrough_Cube_Log.csv
Deleted file: Lookthrough_Cube_overlapped_securities.csv
Deleted file: Lookthrough_Cube_RawLog.csv
Deleted file: Lookthrough_Cube_unprocessed_holdings.csv
Deleted file: Lookthrough_Cube_unprocessed_securities.csv
Deleted file: report.txt
Deleted file: Lookthrough - Cube -  20240105.csv
Deleted file: LookthroughMapping_20240105.csv
replaced S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240105 with S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112 in S:\IT IRSR Shared\RedSwan\Re

## create `Lookthrough - Cube` file
- the code will look for `Lookthrough - Cube` file with the date of `from_date` in the `to_date` working folder. 
- update the date in `MSCI_RM_INDEX_ID` column and `PRICED_SECURITY_NAME` column for ecompass data
- append new week data from sql file in `sql\Lookthrough\lookthrough.sql`
- the code will look for the `PV Report GPF Ext Man` file in current week folder and update positions, so **please run the code after Total Fund Tree is done**, if not needed turn the last variable to `False`
- after manually checking the ecompass data, run the second block of code to turn the excel to csv.

In [2]:
lib.cube_lookthru.create_lookthru_cube(from_date,new_date,True)

FileNotFoundError: [Errno 2] No such file or directory: 'S:\\IT IRSR Shared\\RedSwan\\RedSwan\\Master_bcIMC\\TREE\\Total Fund Tree\\2024\\02\\20240209\\Scale Calculation\\PV Report GPF Ext Man 20240209.xlsx'

In [19]:
lib.cube_lookthru.turn_lookthru_cube_to_csv(new_date)

Converted S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112\Lookthrough - Cube -  20240112.xlsx to csv


## create `LookthroughMapping` file
- the code will look for `Lookthrough - Cube` file with the date of `from_date` in the `to_date` working folder. 
- get rid of records besides ecompass
- append new week data from sql file in `sql\lookthrough\indexCSV.sql`
- the code will look for the `PV Report GPF Ext Man` file in current week folder and update positions, so **please run the code after Total Fund Tree is done**, if not needed turn the last variable to `False`
- `get_lookthru_benchmark_null` function combines look through sql with rm repository data, which brings in the position names in order to search for the correct benchmark id.
- after checking the ecompass data, run the turn to csv code get csv file.

In [14]:
lib.cube_lookthru.create_LookthroughMapping(from_date,new_date,True)

LookthroughMapping created at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112\LookthroughMapping_20240112.xlsx
Deleted file: LookthroughMapping_20240105.xlsx


In [ ]:
lib.cube_lookthru.get_lookthru_benchmark_null()

In [20]:
lib.cube_lookthru.turn_LookthruMapping_to_csv(new_date)

Converted S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2024\20240112\LookthroughMapping_20240112.xlsx to csv


# Look Through Portfolio Tags
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\Portfolio Tag LK\Total Fund Tag Override
- the code will retreieve the liquid data in RMrepository and current week `PV Report GPF Ext Man` report and update to the `from_date` file and save it with `to_date` file name.
- so we need to have the `PV Report GPF Ext Man` file in the folder for `new_date` ready before running the code.

In [ ]:
lib.tag_override.create_look_through_tag_file(from_date,new_date)

# Total Fund Tree
- all code related to operation in Total Fund Tree folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree

## create template folder

In [15]:
lib.total_fund_tree.create_template_folder(from_date,new_date)

Copied folder from S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\03\20240308 to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\03\20240315
Copied to folder at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\03\20240315
Deleted file: overlapped_holdings.csv
Deleted file: TotalFundHierarchy.docx
Deleted file: TotalFundHierarchy.xml
Deleted file: TotalFundHierarchy_20240308.rml
Deleted file: TotalFundHierarchy_20240308_Log.csv
Deleted file: TotalFundHierarchy_20240308_RawLog.csv
Deleted file: unprocessed_holdings.csv
Deleted file: unprocessed_securities.csv
Deleted file: Total_Fund_Tree _20240308.csv
Deleted file: PV Report E0043 Ext Man 20240308.xlsx
Deleted file: PV Report GPF Ext Man 20240308.xlsx
replaced S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\03\20240308 with S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\03\20240315 in S:\IT IRSR Shared\RedSwan\Re

## Update Scale Calculation
- after running the report in risk manager, save `PV Report Liquids External Manager` files in `Scale Calculation` folder
- `create_extMan_PV_reports` will look for file contains `PV Report Liquids External Manager` in the `Scale Calculation` folder under `to_date` working folder.
- add filter to the report and save in `Scale Calculation` folder with the `to_date` date.
- `update_update_mtg_scale_calc` and `update_GPF_scale_calc` will look for respective PV report files in the `Scale Calculation` folder with `to_date` date.
- update the scale calculation files with `from_date` date and save with `to_date` date name.

In [4]:
lib.total_fund_tree.create_extMan_PV_reports(new_date)
lib.total_fund_tree.update_mtg_scale_calc(from_date,new_date)
lib.total_fund_tree.update_GPF_scale_calc(from_date,new_date)

Deleted file: PV Report Liquids External Manager (1).xlsx
Updated PV Report GPF Ext Man 20240112.xlsx in S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Scale Calculation
Deleted file: PV Report Liquids External Manager.xlsx
Updated PV Report E0043 Ext Man 20240112.xlsx in S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Scale Calculation
Deleted file: Scale calculation E0043 20240105.xlsx
Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Scale Calculation\Scale calculation E0043 20240112.xlsx
Deleted file: Scale calculation GPF 20240105.xlsx
Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Scale Calculation\Scale calculation GPF 20240112.xlsx


## Update Total Fund Tree
- manually adjust portfolios in `Total_Fund_Tree` file and save with `to_date` date name.
- `update_total_fund_tree` function will look for `Total_Fund_Tree` with `to_date` date name.
- read scale data from the updated scale files and update the numbers in `Total_Fund_Tree` file
- save updated file and also create a csv file

In [5]:
lib.total_fund_tree.update_total_fund_tree(new_date)

Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Total_Fund_Tree _20240112.xlsx


## Update Total Fund PV Report
- download `PV Report Total Fund` from risk manager and save in `to_date` folder.
- `update_total_fund_pv_report` function will look for `PV Report Total Fund.xlsx` file.
- copy paste `View 1 assetClass by subAsset` sheet. set group level in `View 1 assetClass by subAsset` to level 2, and copied sheet to level 4.
- hide rows from 1 to 19 excepte for row 11, which contains as of date.
- save file with `to_date` name and delete original `PV Report Total Fund.xlsx` file.

In [6]:
lib.total_fund_tree.update_total_fund_pv_report(new_date)

Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\01\20240112\Total Fund PV Report 20240112.xlsx
Deleted file: Total Fund PV Report.xlsx


## GPF Managers Weekly & Monthly MV
- `update_GPF_Managers_MV` function will run the sql query to get the GPF based on `to_date` date.
- paste the data into the `Formuila` sheet in `GPF Managers Weekly & Monthly MV.xlsx` file.
- copy and paste the formated table (`A1:D46`) in `Formula` sheet and paste to new sheet with `to_date` date as name.
- and save the file.

In [5]:
lib.total_fund_tree.update_GPF_Managers_MV(new_date)

S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2024\02\20240229\Queries\GPF Managers Weekly & Monthly MV.xlsx is updated and saved to None


# Total Fund BMK Tree
- all code related to operation in Total Fund BMK Tree folder goes here
- S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree

## create template folder

In [16]:
lib.total_fund_bmk_tree.create_template_folder(from_date,new_date)

Copied folder from S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2024\03\20240308 to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2024\03\20240315
Copied to folder at S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2024\03\20240315
Deleted file: BMK Roll Down 20240308.rml
Deleted file: BMK Roll Down 20240308_Log.csv
Deleted file: BMK Roll Down 20240308_RawLog.csv
Deleted file: BMK Roll Down.docx
Deleted file: BMK Roll Down.xml
Deleted file: BMK Roll Up 20240308.rml
Deleted file: BMK Roll Up 20240308_Log.csv
Deleted file: BMK Roll Up 20240308_RawLog.csv
Deleted file: BMK Roll Up.docx
Deleted file: BMK Roll Up.xml
Deleted file: overlapped_RD_holdings.csv
Deleted file: overlapped_RU_holdings.csv
Deleted file: unprocessed_RD_holdings.csv
Deleted file: unprocessed_RD_securities.csv
Deleted file: unprocessed_RU_holdings.csv
Deleted file: unprocessed_RU_securities.csv
Deleted file: Total_Fund_BMK_Tree_20240308 - RD.c

## Update Total Fund BMK Tree
- manually adjust portfolios in `Total_Fund_BMK_Tree` files and save with `to_date` date in the name.
- `update_total_fund_bmk_tree` function will look for `Total_Fund_BMK_Tree` with `to_date` name.
- the code will grab scale data from the updated scale files in and also the updated PV report in `Total Fund Tree` folder 
- update the scale data and market value in `Total_Fund_BMK_Tree` file
- save the file and also convert it to csv file

In [7]:
lib.total_fund_bmk_tree.update_total_fund_bmk_tree(new_date)

Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2024\01\20240112\Total_Fund_BMK_Tree_20240112 - RD.xlsx and saved csv
Total Fund BMK Tree total MV is 220123901155.5978 and PV report total is 220123901155.59778
Updated S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2024\01\20240112\Total_Fund_BMK_Tree_20240112 - RU.xlsx and saved csv
Total Fund BMK Tree total MV is 220123901155.5978 and PV report total is 220123901155.59778


# Cube Files
- all code related to operation in `CubeFiles` goes here
- S\\bcimcs8\sharedir\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles

## Create template folder
 - the code will create a new folder with the date of `to_date` in the `CubeFiles` folder
 - copy paste all the files
 - update `ProcessStats` file with the date of `to_date` and `RiskVersionName`


In [2]:
weekly = False # True for weekly, False for monthly
lib.cubefiles.create_template_folder(new_date, weekly)

Created folder S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles\Monthly\20231231
Copied file S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Lookthrough for Cube\2023\20231231\LookthroughMapping_20231231.csv to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles\Monthly\20231231
Copied file S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2023\12\20231231\Total_Fund_BMK_Tree_20231231 - RD.csv to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles\Monthly\20231231
Copied file S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund BMK Tree\2023\12\20231231\Total_Fund_BMK_Tree_20231231 - RU.csv to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles\Monthly\20231231
Copied file S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\TREE\Total Fund Tree\2023\12\20231231\Total_Fund_Tree _20231231.csv to S:\IT IRSR Shared\RedSwan\RedSwan\Master_bcIMC\CubeFiles\Monthly\20231231
replaced S&P/LSTA U.S. Leveraged Loan 100 - CAD Hedged with SP.MC.U